In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from feature_eng import ClinicalMetrics, AbsClinicalMetrics, ClinicalSignals, AbsClinicalSignals, NormClinicalMetrics
import pandas as pd
import numpy as np
import argparse
import os
from typing import Dict, List

In [5]:
# Some helper functions

# Audio tasks use an audio.Table while others use a video.Table
audio_tasks = ["BBP_NORMAL", "PA"]

# Used to extract the patient type from the landmark file name
ids = {
    "stroke": ["OP", "S"],
    "healthy": ["N"],
    "als": ["A"]
}

# Defines which metric calculator to use based on the 'metric_type' argument
metric_calc_map = {

    "orofacial_features": ClinicalMetrics,
    "orofacial_nonNormal_features": AbsClinicalMetrics,
    "orofacial_signals": ClinicalSignals,
    'orofacial_nonNormal_signals': AbsClinicalSignals,
    'orofacial_Normal_features': NormClinicalMetrics
}


def compute_metrics(inputs, out_path, metrics_type = 'orofacial_features'):
    """
    Computes the metrics for every file in the data csv
    :param inputs: A dataframe with columns: csv_path, rep_{}_start,
                                                       rep_{}_end
    :param out_path: A path to a csv to store the metric data in
    """
    # Key is task name and value is all repetitions of the task
    metric_frames: Dict[str, List[pd.DataFrame]] = {}
    for index, row in inputs.iterrows():
        # Read all data from the input dataframe
        try:
           
            data_path = row['csv_path']      
            rest_path = row['rest_path']
            data_frame = pd.read_csv(data_path)
            rest_frame = pd.read_csv(rest_path)  
            task_type = row['task']
            subject_id = row['subject']
            subject_type = row['type']
           
        except KeyError:
            print('ERROR!!')
            raise KeyError("Input csv must contain 'csv_path', 'rest_path', 'rest_table_path' \
            'task', 'subject', and 'type' columns. \nFurthermore, 'csv_path' \
            and 'rest_path' must be files on the disk.")
        print("Starting:", data_path, "with", subject_type, "patients")
        rep_ranges = []
        rep_i = 1
        # Grab data about the start and end of repetitions
        while f"rep_{rep_i}_start" in row and f"rep_{rep_i}_end" in row:
            try:
                start_val = row[f"rep_{rep_i}_start"]
                end_val = row[f"rep_{rep_i}_end"]
                rep_ranges.append(range(int(start_val), int(end_val)))
            except ValueError as e:
                pass
            rep_i += 1
        rep_i -= 1


        # Calculate metrics
        metric_calc = metric_calc_map[metrics_type](data_frame, rest_frame)
        metric_frame = None
        for rep_range in rep_ranges:
            metrics = metric_calc.compute_metrics(active_frames=rep_range)
            if metric_frame is None:
                metric_frame = metrics
            else:
                metric_frame = pd.concat([metric_frame, metrics], ignore_index=True)
        rep_nums = list(range(1, rep_i+1))

        # Insert metadata to final csv
        metric_frame.insert(0, 'rep', pd.Series(rep_nums))
        metric_frame.insert(0, 'type', pd.Series(np.full(rep_i, subject_type == "healthy")))
        # Denix changed 
        # metric_frame.insert(0, 'type', pd.Series(np.full(rep_i, int(not(subject_type == "healthy")))))
        metric_frame.insert(0, 'task', pd.Series(np.full(rep_i, task_type)))
        metric_frame.insert(0, 'subject_id', pd.Series(np.full(rep_i, subject_id)))
        try:
            metric_frames[task_type].append(metric_frame)
        except KeyError:
            metric_frames[task_type] = [metric_frame]

    # save data to disk
    for task, metrics in metric_frames.items():
        csv_name = "{}_metrics_from_mobileNet_normalized_{}.csv".format(metrics_type,task)
        
        all_metrics_frame = pd.concat(metrics, ignore_index=True)

        
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        all_metrics_frame.to_csv(os.path.join(out_path, csv_name))
        
    print(f"Saved metrics to {os.path.abspath(out_path)}")
    return metric_frames


def convert_time_to_frames(time_file, landmarks_file):
    """
    Converts audio timestamps into video frames
    :param time_file:
    :param landmarks_file:
    :return:
    """
    data = pd.read_csv(time_file, index_col=None)
    landmarks_DF = pd.read_csv(landmarks_file, index_col=0)
    tmin = data[data.columns[0]].values
    tmax = data[data.columns[-1]].values
    vid_time = landmarks_DF['Time_Stamp (s)'].values[1:]
    positions = np.zeros((len(tmin), 2), dtype=int)
    for k, (mini, maxi) in enumerate(zip(tmin, tmax)):
        positions[k, 0] = int(np.argmin(abs(vid_time - mini)))
        positions[k, 1] = int(np.argmin(abs(vid_time - maxi)))
    new_df = pd.DataFrame()
    new_df['Tstart'] = positions[:, 0]
    new_df['text'] = data['text']
    new_df['Tend'] = positions[:, 1]
    return new_df

In [6]:


def compute_signals(inputs, out_path, metrics_type = 'orofacial_signals'):
    """
    Computes the metrics for every file in the data csv
    :param inputs: A dataframe with columns: csv_path, rep_{}_start,
                                                       rep_{}_end
    :param out_path: A path to a csv to store the metric data in
    """
    # Key is task name and value is all repetitions of the task
    metric_frames: Dict[str, List[pd.DataFrame]] = {}
    for index, row in inputs.iterrows():
        # Read all data from the input dataframe
        try:
            data_path = row['csv_path']      
            rest_path = row['rest_path']
            data_frame = pd.read_csv(data_path)
            rest_frame = pd.read_csv(rest_path)  
            task_type = row['task']
            subject_id = row['subject']
            subject_type = row['type']
            
            if 0:
                data_path = row['csv_path']
                rest_path = row['rest_path']
                data_frame = pd.read_csv(data_path)
                rest_frame = pd.read_csv(rest_path)
                task_type = row['task']
                print(task_type)
                subject_id = row['subject']
               # subject_type = row['type']
        except KeyError:
            raise KeyError("Input csv must contain 'csv_path', 'rest_path', 'rest_table_path' \
            'task', 'subject', and 'type' columns. \nFurthermore, 'csv_path' \
            and 'rest_path' must be files on the disk.")
        print("Starting:", data_path, "with", subject_type, "patients")
        rep_ranges = []
        rep_i = 1
        # Grab data about the start and end of repetitions
        while f"rep_{rep_i}_start" in row and f"rep_{rep_i}_end" in row:
            try:
                start_val = row[f"rep_{rep_i}_start"]
                end_val = row[f"rep_{rep_i}_end"]
                rep_ranges.append(range(int(start_val), int(end_val)))
            except ValueError as e:
                pass
            rep_i += 1
        rep_i -= 1

        # Calculate metrics
        metric_calc = metric_calc_map[metrics_type](data_frame, rest_frame)

        metric_frame = None
        
        for rep_range in rep_ranges:
            
            metrics = metric_calc.compute_metrics(active_frames=rep_range)
            metrics.insert(0, 'Time_Stamp', data_frame.loc[rep_range]['Time_Stamp (s)'].values)
            metrics.insert(0, 'type', pd.Series(np.full(len(metrics), int(not(subject_type == "healthy")))))
            metrics.insert(0, 'task', pd.Series(np.full(len(metrics), task_type)))
            metrics.insert(0, 'rep', pd.Series(np.full(len(metrics), rep_ranges.index(rep_range))))
            metrics.insert(0, 'subject_id', pd.Series(np.full(len(metrics), subject_id)))
            
            
            if metric_frame is None:
                metric_frame = metrics
            else:
                metric_frame = pd.concat([metric_frame, metrics], ignore_index=True)
      

        # Insert metadata to final csv
        
        
        
        try:
            metric_frames[task_type].append(metric_frame)
        except KeyError:
            metric_frames[task_type] = [metric_frame]

    # save data to disk
    for task, metrics in metric_frames.items():
        csv_name = "{}_from_mobileNet_corrected_{}.csv".format(metrics_type,task)
        
        all_metrics_frame = pd.concat(metrics, ignore_index=True)

        
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        all_metrics_frame.to_csv(os.path.join(out_path, csv_name))
        
    print(f"Saved metrics to {os.path.abspath(out_path)}")
    return metric_frames


In [7]:

def save_results(input_csv, output_folder, metric_type):
    analysis_csv = pd.DataFrame()
    files = pd.read_csv(input_csv)
    for i, row in files.iterrows():
        # Construct a dataframe with the information necessary for processing
        landmark_path = os.path.abspath(row["landmarks"])
        landmark_file_name = os.path.basename(landmark_path)
        table_path = os.path.abspath(row["landmarks_table"])
        table_file_name = os.path.basename(table_path)
        rest_path = os.path.abspath(row["rest"])
        rest_file_name = os.path.basename(table_path)
        
        if 0:
            print(landmark_path)
            print()
            print(rest_path)
            print()
            print(table_path)
            print()
            print()

        if not os.path.isfile(landmark_path):
            raise RuntimeError("Landmark path is not a file on disk")
        if not os.path.isfile(rest_path):
            raise RuntimeError("Rest file is not a file on disk")
        if not os.path.isfile(table_path):
            raise RuntimeError("Table file is not a file on disk")

        landmark_file_data = landmark_file_name.split("_")
        
        # Deniz changed May2021
        #subject = landmark_file_data[0]
        subject = row['subject']
        
        
        subject_type = ""
        # Check if a user fits into a known subject type
        for s_type, prefixes in ids.items():
            for prefix in prefixes:
                if prefix in subject:
                    subject_type = s_type
                   
        # Deniz changed May2021
        #task = "_".join(landmark_file_data[2:4])
        task = row['task']
        parsed_data = pd.DataFrame(columns=["csv_path", "type", "rest_path", "subject", "task"])
        parsed_data.loc[0] = 0

        # If it is an audio task, convert the table file to frame numbers
        if 'audio' in table_file_name:
            video_info = convert_time_to_frames(table_path, landmark_path)
        else:
            video_info = pd.read_csv(table_path)


        parsed_data["csv_path"] = landmark_path
        parsed_data["rest_path"] = rest_path
        parsed_data["type"] = subject_type
        parsed_data["subject"] = subject
        parsed_data["task"] = task
        for index, row in video_info.iterrows():
            
            parsed_data[f"rep_{index+1}_start"] = row[video_info.columns[0]]
            parsed_data[f"rep_{index+1}_end"] = row[video_info.columns[-1]] 
        analysis_csv = analysis_csv.append(parsed_data, ignore_index=True, sort=False)

    
    if metric_type == 'orofacial_signals' or metric_type == 'orofacial_nonNormal_signals':
        compute_signals(analysis_csv, output_folder,metric_type);
        
    else:
        compute_metrics(analysis_csv, output_folder,metric_type);

# Main Program 
You need to provide a .csv file with three columns : 'landmarks'  'landmarks_table'   'rest' 


- the 'landmarks' column contains the path to the .csv file that contains the landmark positions for the task under analysis (either 2d or 3d)
- the 'landmarks_table' column cotains the path to the .table file that contains the information about the repetitions either as frames 
  (_video.table) or as time stamps (_audio.table). When using time stamps, there has to be a 'FrameInfoColor.csv' file that contains the 
  relation between time stamps and frame number
- The 'rest' column contains the path to the .csv file that contains the landmark positions for the REST task (either 2d or 3d)


NOTE: There has to be a 'REST' file associated with the subject 
### The program won't continue if the REST file is not provided

### Creat the look up table

### load up the lookup table and run the code for mobileNet Landmarks

In [8]:
# read the df 

input_csv = r'/media/deniz/easystore/facial_tracking_data_2021/mobileNet_3D_lookups.csv'
df= pd.read_csv(input_csv)
df = df.drop('Unnamed: 0', axis=1)
df.head()

,subject,landmarks,landmarks_table,rest,task
0,AF01,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
1,AF03,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
2,AF10,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
3,AF10,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
4,AF11,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN


In [9]:
len(df)

160

In [10]:
metric_type = 'orofacial_Normal_features' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)

Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/mobileNet/AF01_02_20170509_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/mobileNet/AF03_02_20170526_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF10_SBK_20170613/02/LANDMARKS/mobileNet/AF10_02_20170613_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF10_SBK_20170613/02/LANDMARKS/mobileNet/AF10_02_20170613_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF11_SBK_20170928/02/LANDMARKS/mobileNet/AF11_02_20170928_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF11_SBK_20170928/02/LANDMARKS/mobileNet/AF11_02_20170928_NSM_SPREAD_color_landmark

/home/deniz/Code/ClinicalScore/feature_eng.py:350: RuntimeWarning: invalid value encountered in greater
  return z_scores > z_threshold


Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF14_SBK_20190129/02/LANDMARKS/mobileNet/AF14_02_20190129_NSM_KISS_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF14_SBK_20190129/02/LANDMARKS/mobileNet/AF14_02_20190129_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF14_SBK_20190129/02/LANDMARKS/mobileNet/AF14_02_20190129_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_BIGSMILE_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_KISS_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF16_SBK_20190403/02/LANDMARKS/mobileNet/AF16_02_20190403_NSM_SPREAD_color_landma

/home/deniz/Code/ClinicalScore/feature_eng.py:280: UserWarning: Warning: converting a masked element to nan.
  dx[0] = (1 / (2 * h)) * (-3 * feature[0] + 4 * feature[1] - feature[2])
/home/deniz/Code/ClinicalScore/feature_eng.py:281: UserWarning: Warning: converting a masked element to nan.
  dx[1] = (1 / (2 * h)) * (-1 * feature[0] + feature[2])
/home/deniz/.pyenv/versions/reqtxt/lib/python3.6/site-packages/pandas/core/internals/blocks.py:902: UserWarning: Warning: converting a masked element to nan.
  arr_value = np.array(value)


Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/mobileNet/OP05_02_20190719_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/mobileNet/OP05_02_20190719_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_BIGSMILE_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_2019092

Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191107/02/LANDMARKS/mobileNet/OP12_02_20191107_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_BIGSMILE_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_PUCKER_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170

Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_BIGSMILE_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_OPEN_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_PUCKER_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_SPREAD_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/mobileNet/NF17_02_20190613_NSM_BIGSMILE_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021

In [84]:
metric_type = 'orofacial_nonNormal_features' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)

Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/mobileNet/AF01_02_20170509_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/mobileNet/AF03_02_20170526_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF10_SBK_20170613/02/LANDMARKS/mobileNet/AF10_02_20170613_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF10_SBK_20170613/02/LANDMARKS/mobileNet/AF10_02_20170613_NSM_SPREAD_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF11_SBK_20170928/02/LANDMARKS/mobileNet/AF11_02_20170928_NSM_OPEN_color_landmarks3D.csv with als patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/ALS/AF11_SBK_20170928/02/LANDMARKS/mobileNet/AF11_02_20170928_NSM_SPREAD_color_landmark

Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_BIGSMILE_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190830/02/LANDMARKS/mobileNet/OP05_02_20190830_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190926/02/LANDMARKS/mobileNet/OP05_02_20190926_NSM_BIGSMILE_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190926/02/LANDMARKS/mobileNet/OP05_02_20190926_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190

Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_PUCKER_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/mobileNet/OP12_02_20191206_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/mobileNet/S001_02_20170403_NSM_OPEN_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/mobileNet/S001_02_20170403_NSM_SPREAD_color_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S003_TRI_20170728/

Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_PUCKER_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/mobileNet/NF16_02_20190613_NSM_SPREAD_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/mobileNet/NF17_02_20190613_NSM_BIGSMILE_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/mobileNet/NF17_02_20190613_NSM_OPEN_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/mobileNet/NF17_02_20190613_NSM_PUCKER_color_landmarks3D.csv with healthy patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/H

In [24]:
metric_type = 'orofacial_features' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)

In [33]:
output_folder = r"/home/deniz/Code/ClinicalScore/results/test.csv"
df.to_csv(output_folder)

In [ ]:
metric_type = 'orofacial_signals' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'

output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)

In [ ]:
metric_type = 'orofacial_nonNormal_signals' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'

output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)


In [26]:
################# This is the one for data up to now


metric_type = 'orofacial_features' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#finetuned

input_csv = r'/media/deniz/easystore/facial_tracking_data_2021/sbr_3D_lookups.csv'
#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/Test_Table_Feb07.csv"
#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/home/deniz/Code/ClinicalScore/results" #where to store the results
save_results(input_csv, output_folder, metric_type)


/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_SPREAD_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_SPREAD_video.Table


/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/Deniz_SBR/AF03_02_20170526_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/Deniz_SBR/OP05_02_20190719_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/Deniz_SBR/OP05_02_20190719_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/TABLE/OP05_02_20190719_NSM_BIGSMILE_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/Deniz_SBR/OP05_02_20190719_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/Deniz_SBR/OP05_02_20190719_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/TABLE/OP05_02_20190719_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP05_SBK_20190719/02/LANDMARKS/Deniz_SBR/OP

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/Deniz_SBR/S001_02_20170403_NSM_SPREAD_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/Deniz_SBR/S001_02_20170403_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/TABLE/S001_02_20170403_NSM_SPREAD_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/LANDMARKS/Deniz_SBR/S002_02_20170707_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/LANDMARKS/Deniz_SBR/S002_02_20170707_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/TABLE/S002_02_20170707_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/LANDMARKS/Deniz_SBR/S002_0

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S013_TRI_20180302/02/LANDMARKS/Deniz_SBR/S013_02_20180302_NSM_SPREAD_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S013_TRI_20180302/02/LANDMARKS/Deniz_SBR/S013_02_20180302_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S013_TRI_20180302/02/TABLE/S013_02_20180302_NSM_SPREAD_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF00_TRI_20170503/02/LANDMARKS/Deniz_SBR/NF00_02_20170503_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF00_TRI_20170503/02/LANDMARKS/Deniz_SBR/NF00_02_20170503_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF00_TRI_20170503/02/TABLE/NF00_02_20170503_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF00_TRI_20170503/02/LANDMARKS/

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/LANDMARKS/Deniz_SBR/NF09_02_20171012_NSM_SPREAD_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/LANDMARKS/Deniz_SBR/NF09_02_20171012_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF09_SBK_20171012/02/TABLE/NF09_02_20171012_NSM_SPREAD_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF10_SBK_20180212/02/LANDMARKS/Deniz_SBR/NF10_02_20180212_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF10_SBK_20180212/02/LANDMARKS/Deniz_SBR/NF10_02_20180212_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF10_SBK_20180212/02/TABLE/NF10_02_20180212_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF10_SBK_20180212/0

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/Deniz_SBR/NF17_02_20190613_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/Deniz_SBR/NF17_02_20190613_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/TABLE/NF17_02_20190613_NSM_BIGSMILE_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/Deniz_SBR/NF17_02_20190613_NSM_OPEN_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/LANDMARKS/Deniz_SBR/NF17_02_20190613_RST_REST_color_SBR_3D_landmarks3D.csv

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_20190613/02/TABLE/NF17_02_20190613_NSM_OPEN_video.Table


/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF17_TRI_201906

Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/LANDMARKS/Deniz_SBR/OP07_02_20190906_NSM_OPEN_color_SBR_3D_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/Deniz_SBR/OP08_02_20190906_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/Deniz_SBR/OP08_02_20190906_NSM_OPEN_color_SBR_3D_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP08_SBK_20190906/02/LANDMARKS/Deniz_SBR/OP08_02_20190906_NSM_SPREAD_color_SBR_3D_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP09_SBK_20190912/02/LANDMARKS/Deniz_SBR/OP09_02_20190912_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv with stroke patients
Starting: /media/deniz/easystore/facial_tracking_d

In [29]:
out_path = r"/home/deniz/Code/ClinicalScore/results"
os.path.exists(out_path)

True

In [10]:
pwd

'/home/deniz/Code/ClinicalScore'

In [26]:
input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv"

df= pd.read_csv(input_csv)
df.head()

FileNotFoundError: [Errno 2] File b'/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv' does not exist: b'/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv'

In [16]:
df.loc[0][0]

'/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv'

In [7]:
def change_path(df_cell):

    df_cell = df_cell.replace("E:\FaceData_", "\\Users\denizjafari\documents\CODE\ClinicalScore\FaceData_")
    df_cell = df_cell.replace("\\", "/")
    return df_cell

In [8]:
# change the path to the correct path on my MAC 
try:
    df['landmarks'] = df['landmarks'].apply(lambda x: change_path(x))
    df['landmarks_table'] = df['landmarks_table'].apply(lambda x: change_path(x))
    df['rest'] = df['rest'].apply(lambda x: change_path(x))
except:
    pass

# Write the results into csv
df.to_csv('ALS_New_Table.csv')
try:
    df = df.drop(['Unnamed: 0'], axis=1)
except:
    pass

df.head()

,landmarks,landmarks_table,rest
0,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
1,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
2,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
3,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...
4,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...,/Users/denizjafari/documents/CODE/ClinicalScor...


In [10]:
df_test = df
df_test.to_csv('Test_Table_Feb07.csv')
#df_test

In [11]:
metric_type = 'orofacial_features' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#finetuned

input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/Test_Table_Feb07.csv"
#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/results/TEST" #where to store the results
save_results(input_csv, output_folder, metric_type)



/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_BBP_NORMAL_audio.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_OPEN_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_NSM_OPEN_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_NSM_SPREAD_landmarksFiltered3D.csv

/Users/denizjafari/documents/COD

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/A007_02_BBP_NORMAL_audio.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_OPEN_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/A007_02_NSM_OPEN_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_SPREAD_landmarksFiltered3D.csv

/Users/denizjafari/documents/COD

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF18/landmarks_finetuned_w15/AF18_02_NSM_SPREAD_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF18/landmarks_finetuned_w15/AF18_02_RST_REST3_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF18/AF18_02_NSM_SPREAD_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/Healthy-Control/old/NF01/landmarks_finetuned_w15/NF01_02_NSM_OPEN_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/Healthy-Control/old/NF01/landmarks_finetuned_w15/NF01_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/Healthy-Control/old/NF01/NF01_02_NSM_OPEN_video.Table
/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/Healthy-Control/old/NF01/landmarks_finetuned_w15/NF01_02_NSM_SPREAD_landmark

/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/feature_eng.py:394: RuntimeWarning: invalid value encountered in greater
  return z_scores > z_threshold


Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A005/landmarks_finetuned_w15/A005_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A005/landmarks_finetuned_w15/A005_02_NSM_SPREAD_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_OPEN_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A007/landmarks_finetuned_w15/A007_02_NSM_SPREAD_landmarksFiltered3D.csv with als patients
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/new/AF13/landmarks_finetuned_w15/A01

In [27]:
print(input_csv)
os.path.abspath(input_csv)

/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv


'/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_vs_1Control_finetuned.csv'

In [ ]:
break

In [63]:
# Try with mini lookup table of 1 subject

metric_type = 'orofacial_signals' #Which Metric Type to Compute, available types='stroke', 'als', or 'all'


#finetuned

input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/Test_Table.csv"
#input_csv = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/ALS_New_Table.csv" #csv file with landmarks and .TABLE info
output_folder = r"/Users/denizjafari/documents/CODE/ClinicalScore/ClinicalScore/results/TEST" #where to store the results
save_results(input_csv, output_folder, metric_type)





/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_RST_REST_landmarksFiltered3D.csv

/Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/A001_02_BBP_NORMAL_audio.Table
Starting: /Users/denizjafari/documents/CODE/ClinicalScore/FaceData_/video_data/ALS/old/A001/landmarks_finetuned_w15/A001_02_BBP_NORMAL_landmarksFiltered3D.csv with als patients
range(26, 114)
time length:  1336
time length:  88
signal length:  88

range(156, 238)
time length:  1336
time length:  82
signal length:  82

range(285, 367)
time length:  1336
time length:  82
signal length:  82

range(408, 499)
time length:  1336
time length:  91
signal length:  91

range(543, 633)
time length:  1336
time length:  90
signal length:  90

range(678, 762)
time length:  1336
time length:  84
s

In [ ]:
break 

In [ ]:
### THE END!

In [4]:
a = ["O_MAX", "O_MIN", "O_AVG","O_RANGE" ,"O_PATH", "vO_MAX", "vO_MIN", "vO_AVG", "aO_MAX", "aO_MIN", "aO_AVG","jO_MAX", "jO_MIN", "jO_AVG",
                       "W_MAX", "W_MIN", "W_AVG","W_RANGE" ,"W_PATH", "vW_MAX", "vW_MIN", "vW_AVG", "aW_MAX", "aW_MIN", "aW_AVG","jW_MAX", "jW_MIN", "jW_AVG",
                       "LL_MAX", "LL_MIN", "LL_AVG","LL_RANGE" ,"LL_PATH", "vLL_MAX", "vLL_MIN", "vLL_AVG", "aLL_MAX", "aLL_MIN", "aLL_AVG","jLL_MAX", "jLL_MIN", "jLL_AVG",
                       "RC_MAX", "RC_MIN", "RC_AVG","RC_RANGE" ,"RC_PATH", "vRC_MAX", "vRC_MIN", "vRC_AVG", "aRC_MAX", "aRC_MIN", "aRC_AVG","jRC_MAX", "jRC_MIN", "jRC_AVG",
                       "LC_MAX", "LC_MIN", "LC_AVG","LC_RANGE" ,"LC_PATH", "vLC_MAX", "vLC_MIN", "vLC_AVG", "aLC_MAX", "aLC_MIN", "aLC_AVG","jLC_MAX", "jLC_MIN", "jLC_AVG",
                       "RJ_MAX", "RJ_MIN", "RJ_AVG","RJ_RANGE" ,"RJ_PATH", "vRJ_MAX", "vRJ_MIN", "vRJ_AVG", "aRJ_MAX", "aRJ_MIN", "aRJ_AVG","jRJ_MAX", "jRJ_MIN", "jRJ_AVG",
                       "LJ_MAX", "LJ_MIN", "LJ_AVG","LJ_RANGE" ,"LJ_PATH", "vLJ_MAX", "vLJ_MIN", "vLJ_AVG", "aLJ_MAX", "aLJ_MIN", "aLJ_AVG","jLJ_MAX", "jLJ_MIN", "jLJ_AVG",
                       "RCLC_diff", "RJLJ_diff","C_RCLC","P_RCLC","C_RALA","P_RALA","R_RALA","C_RJLJ","P_RJLJ","C_JRALA","P_JRALA","R_JRALA","e_AVG","e_RANGE",
                       "tA_Max","tA_MIN","tA_AVG","tA_RANGE","rA_Max","rA_MIN","rA_AVG","rA_RANGE","lA_Max","lA_MIN","lA_AVG","lA_RANGE","A_diff",
                       "tJA_Max","tJA_MIN","tJA_AVG","tJA_RANGE","rJA_Max","rJA_MIN","rJA_AVG","rJA_RANGE","lJA_Max","lJA_MIN","lJA_AVG","lJA_RANGE","JA_diff"
                       ]

len(a)

138

In [49]:
c = np.array([1, 2, 3, 4, 5])
c.shape

(5,)

In [ ]:
np.linalg.norm(a[0,:] - a[1, :], axis=0)

In [5]:
b =  pd.DataFrame(columns=a)
b

,O_MAX,O_MIN,O_AVG,O_RANGE,O_PATH,vO_MAX,vO_MIN,vO_AVG,aO_MAX,aO_MIN,...,tJA_RANGE,rJA_Max,rJA_MIN,rJA_AVG,rJA_RANGE,lJA_Max,lJA_MIN,lJA_AVG,lJA_RANGE,JA_diff


In [6]:
b.loc[0] = 0
b

,O_MAX,O_MIN,O_AVG,O_RANGE,O_PATH,vO_MAX,vO_MIN,vO_AVG,aO_MAX,aO_MIN,...,tJA_RANGE,rJA_Max,rJA_MIN,rJA_AVG,rJA_RANGE,lJA_Max,lJA_MIN,lJA_AVG,lJA_RANGE,JA_diff
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
b.loc[0][['O_MAX']] = 1
b

,O_MAX,O_MIN,O_AVG,O_RANGE,O_PATH,vO_MAX,vO_MIN,vO_AVG,aO_MAX,aO_MIN,...,tJA_RANGE,rJA_Max,rJA_MIN,rJA_AVG,rJA_RANGE,lJA_Max,lJA_MIN,lJA_AVG,lJA_RANGE,JA_diff
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
a = [4,3,5,67,3,6]

for num in a:
    print(a.index(num))
    

0
1
2
3
1
5


In [ ]:
np.vstack([[1,1,1,1,1],[3,3,3,3,3],[2,2,2,2,2]]).transpose()

In [61]:
df = pd.DataFrame(index=range(len(c)), columns=a)
df = df.fillna(0)
df.loc[0:5, ['O_MAX','O_MIN','O_AVG']] = np.array([[1,1,1,1,1],[3,3,3,3,3],[2,2,2,2,2]]).transpose()
df.loc[range(2)]['O_MAX'].values

array([1, 1])

In [43]:
np.array([[1,1,1,1,1],[3,3,3,3,3],[2,2,2,2,2]]).transpose()

array([[1, 3, 2],
       [1, 3, 2],
       [1, 3, 2],
       [1, 3, 2],
       [1, 3, 2]])